# Projet : Systeme de recommandation d'images

## Presentation

Dans ce projet, vous allez construire un **systeme de recommandation d'images** qui suggere des images aux utilisateurs en fonction de leurs preferences. Ce projet met en pratique toutes les competences acquises lors des seances pratiques : analyse de donnees, visualisation, regroupement, classification et apprentissage automatique.

**Duree** : 3 seances pratiques
**Taille de l'equipe** : 2-3 etudiants
**Livrables** :
1. Un notebook Jupyter (`Nom1_Nom2_[Nom3].ipynb`)
2. Un rapport de synthese de 4 pages (PDF)

---

## Objectifs d'apprentissage

En completant ce projet, vous serez capable de :
- Automatiser la collecte de donnees a partir de sources web
- Extraire et traiter les metadonnees d'images
- Appliquer des algorithmes de regroupement pour analyser les caracteristiques des images
- Construire des profils de preferences utilisateur
- Implementer un algorithme de recommandation
- Visualiser efficacement les donnees
- Ecrire des tests complets pour votre systeme

---

## Architecture du projet

Le systeme est compose de 7 taches interconnectees :

```
┌─────────────────────────────────────────────────────────────────┐
│               SYSTEME DE RECOMMANDATION D'IMAGES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 1. Collecte  │───▶│ 2. Etiquetage│───▶│ 3. Analyse   │       │
│  │ de donnees   │    │ & Annotation │    │ de donnees   │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────────────────────────────────────────────┐       │
│  │          Fichiers JSON (Stockage des metadonnees)     │       │
│  └──────────────────────────────────────────────────────┘       │
│        │                    │                   │                │
│        ▼                    ▼                   ▼                │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐       │
│  │ 4. Visuali-  │    │ 5. Systeme de│    │ 6. Tests     │       │
│  │ sation       │    │ recommandat. │    │              │       │
│  └──────────────┘    └──────────────┘    └──────────────┘       │
│                             │                                    │
│                             ▼                                    │
│                    ┌──────────────┐                              │
│                    │ 7. Rapport   │                              │
│                    │ de synthese  │                              │
│                    └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

![Architecture](../../images/Project-Architecture.png "Architecture")

---

## Projet partie 1

---

### Tache 1 : Collecte de donnees

#### Objectif
Collecter au moins **100 images sous licence ouverte** avec leurs metadonnees.

#### Ce que vous devez faire

1. **Creer une structure de dossiers** :
   ```
   projet/
   ├── images/           # Images telechargees
   ├── data/             # Fichiers de metadonnees JSON
   └── projet.ipynb      # Votre notebook
   ```

2. **Trouver des sources d'images** (une ou plusieurs) :
   - [Wikimedia Commons](https://commons.wikimedia.org/) - Utilisez des requetes SPARQL (comme dans le TP 1)
   - [Unsplash API](https://unsplash.com/developers) - API gratuite pour des images de haute qualite
   - [Pexels API](https://www.pexels.com/api/) - Photos libres de droits
   - [Flickr API](https://www.flickr.com/services/api/) - Images Creative Commons

3. **Telecharger les images par programme** en utilisant les techniques du TP 1, Exercice 6

4. **Extraire et sauvegarder les metadonnees** de chaque image :
   - Nom du fichier image
   - Dimensions de l'image (largeur, hauteur)
   - Format du fichier (.jpg, .png, etc.)
   - Taille du fichier (en Ko)
   - URL source
   - Informations de licence
   - Donnees EXIF (si disponibles) : modele d'appareil photo, date de prise de vue, etc.

#### Resultat attendu
- Dossier `images/` contenant 100+ images
- `data/images_metadata.json` contenant les metadonnees de toutes les images

#### Conseils
- Utilisez `PIL` pour obtenir les dimensions de l'image
- Utilisez `os.path.getsize()` pour obtenir la taille du fichier
- Utilisez l'extraction EXIF (voir TP 2, Exercice 2)
- Stockez les metadonnees sous forme de liste de dictionnaires au format JSON

---

## Tâche 1 — Collecte de données : téléchargement des images
On récupère la liste des photos depuis l'API `jmail.world`, puis on télécharge les 100 premières dans `images/`.

In [2]:
import requests, time
import json
from pathlib import Path

# ── Config ─────────────────────────────────────────────────────
LIMIT = 100
API_URL = "https://jmail.world/api/photos?newOnly=false"
CDN_BASE = "https://assets.getkino.com/photos-deboned"
IMAGES_DIR = Path("images")
DATA_DIR = Path("data")
IMAGES_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ── 1. Récupérer la liste des photos depuis l'API ─────────────
session = requests.Session()
resp = session.get(API_URL, timeout=30)
resp.raise_for_status()
all_photos = resp.json()["photos"]

# Filtrer : garder seulement les images AVEC source_url
available_photos = [p for p in all_photos if p.get("source_url")]
print(f"API : {len(all_photos)} photos total, {len(available_photos)} avec source_url")

# ── 2. Télécharger jusqu'à avoir LIMIT images réussies ────────
downloaded = 0
successful_photos = []
attempted = 0

for photo in available_photos:
    if downloaded >= LIMIT:
        break
    
    attempted += 1
    filename = photo["original_filename"]
    dest = IMAGES_DIR / filename
    
    # Si l'image existe déjà, on la compte comme réussie
    if dest.exists():
        downloaded += 1
        successful_photos.append(photo)
        continue
    
    url = f"{CDN_BASE}/{filename}"
    try:
        r = session.get(url, stream=True, timeout=60)
        r.raise_for_status()
        with open(dest, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        downloaded += 1
        successful_photos.append(photo)
        if downloaded % 20 == 0:
            print(f"  [{downloaded}/{LIMIT} downloaded, {attempted} tried]…")
    except Exception as e:
        print(f"  ✗ {filename}: {e}")
    
    time.sleep(0.05)

# ── 3. Sauvegarder SEULEMENT les métadonnées des images téléchargées ──
metadata_file = DATA_DIR / "image_metadata.json"
with open(metadata_file, "w", encoding="utf-8") as f:
    json.dump(successful_photos, f, ensure_ascii=False, indent=2)

print(f"\n✓ {downloaded} images dans {IMAGES_DIR}/ (essayé {attempted} photos)")
print(f"✓ Métadonnées de {len(successful_photos)} images sauvegardées dans {metadata_file}")

API : 18316 photos total, 5527 avec source_url
  ✗ EFTA00004157-3.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-3.png
  ✗ EFTA00004157-5.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-5.png
  ✗ EFTA00004157-7.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00004157-7.png
  ✗ EFTA00005381-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005381-1.png
  ✗ EFTA00005386-78.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005386-78.png
  ✗ EFTA00005569-1.png: 404 Client Error: Not Found for url: https://assets.getkino.com/photos-deboned/EFTA00005569-1.png
  [20/100 downloaded, 26 tried]…
  [40/100 downloaded, 46 tried]…
  [60/100 downloaded, 66 tried]…
  [80/100 downloaded, 86 tried]…
  [100/100 downloaded, 106 tried]…

✓ 100 images dans images/ (essayé 106 photos)
✓ Mét

In [3]:
!pip install Pillow exifread

In [23]:
from PIL import Image
import json
import os
import pandas as pd

modeListe = []
tailleListe = []
sizeCategoryListe = []
orientationListe = []

# Load JSON data from a file
data = json.load(open("data/image_metadata.json"))

# Convert JSON data to a pandas DataFrame
dataframe = pd.json_normalize(data)

for index, row in dataframe.iterrows():
    
    image_path = "images/" + row["id"]
    
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            modeListe.append(img.mode)
            tailleListe.append(str(round(os.path.getsize(image_path) / 1024, 2)) + " ko")
            
            # Size classification based on maximum dimension
            width, height = img.size
            max_dimension = max(width, height)
            
            if max_dimension < 500:
                sizeCategoryListe.append("thumbnail")
            elif max_dimension <= 1500:
                sizeCategoryListe.append("medium")
            else:
                sizeCategoryListe.append("large")
            
            # Orientation detection
            if width > height:
                orientationListe.append("landscape")
            elif height > width:
                orientationListe.append("portrait")
            else:
                orientationListe.append("square")
    else:
        paletteListe.append("Image not found")
        modeListe.append("Image not found")
        tailleListe.append("Image not found")
        sizeCategoryListe.append("unknown")
        orientationListe.append("unknown")
        
dataframe["mode"] = modeListe
dataframe["size"] = tailleListe
dataframe["size_category"] = sizeCategoryListe
dataframe["orientation"] = orientationListe
dataframe.to_json(path_or_buf="data/metadata.json", orient="columns")


In [ ]:
tag = ["album_photo", "extérieur", "illisible", "intérieur", "aJE", "jour", "nuit"]

photo_tags = [
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[3], tag[4], tag[6]],
    [tag[3]],
    [tag[3]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[3], tag[4], tag[6]],
    [tag[3], tag[4], tag[6]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[4], tag[5]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[1], tag[5]],
    [tag[3]],
    [tag[3]],
    [tag[3]],
    [tag[1], tag[5]],
    [tag[3], tag[4]],
    [tag[3], tag[4]],
    [tag[1], tag[4], tag[5]],
    [tag[3]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[2]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[6]],
    [tag[1], tag[4], tag[5]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[2]],
    [tag[0], tag[4]],
    [tag[0]],
    [tag[2]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0]],
    [tag[0], tag[4]],
    [tag[0]],
    [tag[0]],
    [tag[2]],
]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3340153426.py, line 4)